# 深度学习课程设计 图像描述生成 结题报告 


## 准备工作:环境配置

- 运行系统版本：
Kylin Linux Advanced Server / WSL2 Ubuntu 22.04.1 
- python版本
python3.7.16
- 框架版本
torch==1.10.2
torchvision==0.11.3
- openjdk版本
openjdk version "1.8.0_392"

下载java1.8 用于评估指标


In [ ]:
%%bash
# 根据系统选择
yum -y list java*
apt-cache search openjdk

In [ ]:
%%bash
conda create -n nndlexp python=3.7.16
pip install -r requirments.txt
pip install -r nnlgeval/requirments.txt

In [ ]:
%%bash
# 评估资源下载
python3 nlgeval/resource_download.py

In [ ]:
# nltk语料库下载
import nltk

nltk.download('punkt')

## 实验： 

- 我们使用了老师提供的deepfashion-mini数据集，
    - 训练集包含10155条图像和对应的文本
    - 测试集包含2538条图像和对应的文本
- 实现了以下模型结构
    - CNN网格表示+GRU
    - CNN网格表示+Transformer
- 实现了以下评测标准
    - BLEU
    - METEOR
    - ROUGE
    - CIDEr
    - SPICE

### 读取配置

[代码实现](utils/config.py)

我们实现了一个配置类用于存储所有的超参数以及模型配置
```json
{
    "use_model_type": "CNN_Transformer",
    "num_epoch": 4,
    "batch_size": 64,
    "eval_batch_size": 64,
    "encoder_lr": 2e-4,
    "decoder_lr": 5e-4,
    "beam_k": 5,
    "CNN_GRU": {
        "img_dim": 2048,
        "cap_dim": 512,
        "hidden_size": 512,
        "num_layers": 1
    },
    "CNN_Transformer": {
        "embed_size": 32,
        "num_head": 8,
        "dim_ff": 512,
        "num_encoder": 6,
        "num_decoder": 6
    },
    "vocab_size": 109,
    "max_len": 25,
    "dataset_path": "data/deepfashion-mini",
    "vocab_path": "data/deepfashion-mini/vocab.json",
    "model_checkpoint_path": "checkpoints/12-25_23-08CNN_Transformer/model_checkpoint2.pth",
    "saved_device_type": null
}
```

In [ ]:
from utils.config import Config

config = Config()
config.read_config('config.json')  # 读取参数，打印参数

### 数据预处理

[代码实现](utils/data_loader.py)

In [ ]:
from utils.data_loader import get_dataloader, data_preprocess

# 数据加载
data_preprocess(config.dataset_path)  # 只需要运行一次
train_loader, test_loader, eval_loader = get_dataloader(config.dataset_path,
                                                        config.batch_size,
                                                        config.eval_batch_size,
                                                        workers=0)


### 模型训练

[代码实现](utils/trainer.py)

模型的训练可以作为

In [ ]:
import torch 
from utils.trainer import train, evaluate, evaluation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 训练函数，epoch batchsize等都在config.json定义
train(train_loader, test_loader, config)